In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon
import statsmodels

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'
ais_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/AIS_data/all_centers_ais_data.csv'
main_diagnosis_complement_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/main_dx_complement.xlsx'


In [ ]:
df = pd.read_excel(data_path)
main_diagnosis_complement_df = pd.read_excel(main_diagnosis_complement_data_path, header=1)


In [ ]:
ais_df = pd.read_csv(ais_data_path)
ais_df.drop(columns=['Name', 'Vorname'], inplace=True)
df = df.merge(ais_df, how='left', on='SNZ Einsatz Nr.  (Einsatz)')

In [ ]:
main_diagnosis_complement_df = main_diagnosis_complement_df[['SNZ Einsatz Nr.  (Einsatz)', 'Einteilung (Hauptdiagnose)', 'Kategorie (Hauptdiagnose)']]
main_diagnosis_complement_df['Einteilung (Hauptdiagnose)'] = main_diagnosis_complement_df['Einteilung (Hauptdiagnose)'].replace(['Keine Diagnose', 'Unbekannt'], np.nan)
df['Einteilung (Hauptdiagnose)'] = df['Einteilung (Hauptdiagnose)'].replace(['Keine Diagnose', 'Unbekannt'], np.nan)
df['Kategorie (Hauptdiagnose)'] = df['Kategorie (Hauptdiagnose)'].replace(['Keine Diagnose', 'Unbekannt'], np.nan)

main_diagnosis_complement_df = main_diagnosis_complement_df.rename(columns={
    'Einteilung (Hauptdiagnose)': 'completed_main_diagnosis',
    'Kategorie (Hauptdiagnose)': 'completed_main_diagnosis_category'
})
main_diagnosis_complement_df = main_diagnosis_complement_df.dropna()
main_diagnosis_complement_df = main_diagnosis_complement_df.drop_duplicates()

# find all 'SNZ Einsatz Nr.  (Einsatz)' with nans in 'Einteilung (Hauptdiagnose)' column in df
missing_dx_ids = df[df['Einteilung (Hauptdiagnose)'].isna()]['SNZ Einsatz Nr.  (Einsatz)'].unique()
main_diagnosis_complement_df = main_diagnosis_complement_df[main_diagnosis_complement_df['SNZ Einsatz Nr.  (Einsatz)'].isin(missing_dx_ids)]

df = df.merge(main_diagnosis_complement_df, how='left', on='SNZ Einsatz Nr.  (Einsatz)')
# replace nan in 'Einteilung (Hauptdiagnose)' with 'completed_main_diagnosis' 
df['Einteilung (Hauptdiagnose)'] = df['Einteilung (Hauptdiagnose)'].fillna(df['completed_main_diagnosis'])
df['Kategorie (Hauptdiagnose)'] = df['Kategorie (Hauptdiagnose)'].fillna(df['completed_main_diagnosis_category'])
# drop 'completed_main_diagnosis' column
df = df.drop(columns=['completed_main_diagnosis']) 

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
df['Einteilung (Hauptdiagnose)'].value_counts()

In [ ]:
# find number of nans in column
df['Einteilung (Hauptdiagnose)'].isna().sum()

In [ ]:
df['Diagnose präklinisch'].unique()

In [ ]:
df['GCS (Erstbefund)'].value_counts()

In [ ]:
# number of nans in GCS
print(f'Number of nans in GCS: {df["GCS (Erstbefund)"].isna().sum()}')
assert df['GCS (Erstbefund)'].isna().sum() == 0


In [ ]:
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})

## Overall GCS distribution: medical vs trauma

In [ ]:
df[(df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].describe()

In [ ]:
df[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].describe()

Uncorrected

In [ ]:
from statsmodels.stats.nonparametric import rank_compare_2indep

In [ ]:
res = rank_compare_2indep(df[(df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].values,
                    df[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].values)

In [ ]:
res.pvalue

with correction for age / sex

In [ ]:
# encode eineteilung
df['Einteilung_encoded'] = df['Einteilung (Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']], distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)

In [ ]:
res_prob_med.summary()

with correction for age / sex / AIS head - but adjusting for AIS does not really make sense

In [ ]:
temp_df = df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']]
# within medical patients, set AIS Head to 0
temp_df[(temp_df['Einteilung_encoded'] == 0)]['AIS Head'] = 0
temp_df.dropna(inplace=True)

mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']], distr='probit')

res_prob_med = mod_prob_med.fit(maxiter=10000)
res_prob_med.summary()


In [ ]:
# plot boxplot of GCS according to main diagnosis
fig = plt.figure(figsize=(10, 6))
ax = sns.boxplot(x='Einteilung (Hauptdiagnose)', y='GCS (Erstbefund)', data=df[df['Einteilung (Hauptdiagnose)'] != 'Keine Diagnose'],
                 showfliers=False, hue='Einteilung (Hauptdiagnose)', palette='viridis', legend=False)

# add annotation of p-value (line between boxes)
# get the x and y coordinates
x1, x2 = 0, 1
y, h, col = df['GCS (Erstbefund)'].max() + 0.4, 0.3, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h+0.1, f'p={res.pvalue:.3f}', ha='center', va='bottom', color=col)

ax.set_ylim(3, 17)
plt.ylabel('GCS')

# don't allow y_ticks > 15
plt.yticks(range(3, 16, 2))

plt.xticks(rotation=45)
# set x ticks
plt.xticks(ticks=[0, 1], labels=['Non-Trauma', 'Trauma'])
plt.xlabel('')

plt.show()

In [ ]:
# fig.savefig('/Users/jk1/Downloads/gcs_intubation.png', dpi=600, bbox_inches='tight')

In [ ]:
# build table with median (IQR) of GCS for each main diagnosis, and p-value

results_df = pd.DataFrame(columns=['Medical', 'Trauma', 'p-value'])
medical_df = df[(df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)']
trauma_df = df[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)']
results_df.loc['n'] = [len(medical_df), len(trauma_df), '']
results_df.loc['GCS (median/IQR)'] = [f'{medical_df.median()} ({medical_df.quantile(0.25)}-{medical_df.quantile(0.75)})',
                        f'{trauma_df.median()} ({trauma_df.quantile(0.25)}-{trauma_df.quantile(0.75)})',
                        res.pvalue]



In [ ]:
results_df

In [ ]:
# results_df.to_csv('/Users/jk1/Downloads/gcs_intubation.csv')

### GCS distribution: non-trauma vs trauma (excluding burns) 

In [ ]:
no_burns_df = df[df['Kategorie (Hauptdiagnose)'] != 'Verbrennung']
no_burns_df.shape, df.shape

Uncorrected

In [ ]:
no_burns_res = rank_compare_2indep(no_burns_df[(no_burns_df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (no_burns_df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].values,
                    no_burns_df[(no_burns_df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (no_burns_df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)'].values)

In [ ]:
no_burns_res.pvalue

With correction

In [ ]:
no_burns_temp_df = no_burns_df[['GCS (Erstbefund)', 'Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
no_burns_temp_df.dropna(inplace=True)

no_burns_mod_prob_med = OrderedModel(no_burns_temp_df['GCS (Erstbefund)'],
                            no_burns_temp_df[['Einteilung_encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']],
                            distr='probit')
no_burns_res_prob_med = no_burns_mod_prob_med.fit(maxiter=10000)

In [ ]:
no_burns_res_prob_med.summary()

In [ ]:
# fig.savefig('/Users/jk1/Downloads/gcs_intubation_no_burns.png', dpi=600, bbox_inches='tight')

In [ ]:
# plot boxplot of GCS according to main diagnosis
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
# add horizontal lines
sns.set_style("whitegrid")
sns.set_context("notebook")

sns.boxplot(x='Einteilung (Hauptdiagnose)', y='GCS (Erstbefund)', data=df[df['Einteilung (Hauptdiagnose)'] != 'Keine Diagnose'],
                 showfliers=False, hue='Einteilung (Hauptdiagnose)', palette='viridis', legend=False, ax=axs[0])

# add annotation of p-value (line between boxes)
# get the x and y coordinates
x1, x2 = 0, 1
y, h, col = df['GCS (Erstbefund)'].max() + 0.4, 0.3, 'k'
axs[0].plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
axs[0].text((x1+x2)*.5, y+h+0.1, f'p={res.pvalue:.3f}', ha='center', va='bottom', color=col)

axs[0].set_ylim(3, 17.5)
axs[0].set_ylabel('GCS')

# don't allow y_ticks > 15
axs[0].set_yticks(range(3, 16, 2))

# set x ticks
axs[0].set_xticks(ticks=[0, 1], labels=['Non-Trauma', 'Trauma'], rotation=45)
axs[0].set_xlabel('')

# plot boxplot of GCS according to main diagnosis
sns.boxplot(x='Einteilung (Hauptdiagnose)', y='GCS (Erstbefund)', data=no_burns_df[no_burns_df['Einteilung (Hauptdiagnose)'] != 'Keine Diagnose'],
                    showfliers=False, hue='Einteilung (Hauptdiagnose)', palette='viridis', legend=False, ax=axs[1])

# add annotation of p-value (line between boxes)
# get the x and y coordinates
x1, x2 = 0, 1
y, h, col = no_burns_df['GCS (Erstbefund)'].max() + 0.4, 0.3, 'k'
axs[1].plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
axs[1].text((x1+x2)*.5, y+h+0.1, f'p={no_burns_res.pvalue:.3f}', ha='center', va='bottom', color=col)

axs[1].set_ylim(3, 17.5)
axs[1].set_ylabel('GCS')

# don't allow y_ticks > 15
axs[1].set_yticks(range(3, 16, 2))

# set x ticks
axs[1].set_xticks(ticks=[0, 1], labels=['Non-Trauma', 'Trauma'], rotation=45)
axs[1].set_xlabel('')

# add number of patients  over each boxplot
axs[0].text(0, 16.5, f'n={len(medical_df)}', ha='center', va='bottom')
axs[0].text(1, 16.5, f'n={len(trauma_df)}', ha='center', va='bottom')
axs[1].text(0, 16.5, f'n={len(no_burns_df[(no_burns_df["Einteilung (Hauptdiagnose)"] == "Krankheit") & (no_burns_df["GCS (Erstbefund)"].notna())])}', ha='center', va='bottom')
axs[1].text(1, 16.5, f'n={len(no_burns_df[(no_burns_df["Einteilung (Hauptdiagnose)"] == "Unfall") & (no_burns_df["GCS (Erstbefund)"].notna())])}', ha='center', va='bottom')
plt.show()

In [ ]:
# fig.savefig('/Users/jk1/Downloads/gcs_intubation_combined_figure.png', dpi=600, bbox_inches='tight')

In [ ]:
no_burns_df['burns'] = 'excluded'
df['burns'] = 'included'
# merge dataframes
merged_df = pd.concat([df, no_burns_df], ignore_index=True)

# plot boxplot of GCS according to main diagnosis with burn exclusion on x axis an trauma / non trauma as hue
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='burns', y='GCS (Erstbefund)', data=merged_df[merged_df['Einteilung (Hauptdiagnose)'] != 'Keine Diagnose'],
                 showfliers=False, hue='Einteilung (Hauptdiagnose)', palette='viridis', legend=True, ax=ax)

# add annotation of p-value (line between boxes)
# get the x and y coordinates
x1, x2 = 0, 1
y, h, col = merged_df['GCS (Erstbefund)'].max() + 0.4, 0.3, 'k'
ax.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
ax.text((x1+x2)*.5, y+h+0.1, f'p={no_burns_res.pvalue:.3f}', ha='center', va='bottom', color=col)
ax.set_ylim(3, 17)
ax.set_ylabel('GCS')
# don't allow y_ticks > 15
ax.set_yticks(range(3, 16, 2))
# set x ticks
ax.set_xticks(ticks=[0, 1], labels=['Including burn patients', 'Excluding burn patients'])
ax.set_xlabel('')
plt.show()

In [ ]:
# build table with median (IQR) of GCS for each main diagnosis, and p-value

no_burns_results_df = pd.DataFrame(columns=['Medical', 'Trauma', 'p-value'])
no_burns_medical_df = no_burns_df[(no_burns_df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (no_burns_df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)']
no_burns_trauma_df = no_burns_df[(no_burns_df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (no_burns_df['GCS (Erstbefund)'].notna())]['GCS (Erstbefund)']
no_burns_results_df.loc['n'] = [len(no_burns_medical_df), len(no_burns_trauma_df), '']
no_burns_results_df.loc['GCS (median/IQR)'] = [f'{no_burns_medical_df.median()} ({no_burns_medical_df.quantile(0.25)}-{no_burns_medical_df.quantile(0.75)})',
                        f'{no_burns_trauma_df.median()} ({no_burns_trauma_df.quantile(0.25)}-{no_burns_trauma_df.quantile(0.75)})',
                        no_burns_res.pvalue]

In [ ]:
no_burns_results_df

In [ ]:
# no_burns_results_df.to_csv('/Users/jk1/Downloads/gcs_intubation_no_burns.csv')

### Evaluation of subgroups in medical/trauma

Test: Kruskal-Wallis

In [ ]:
df['Kategorie (Hauptdiagnose)'].unique()

In [ ]:
df['Kategorie (Hauptdiagnose)'].isna().sum()

#### Medical

In [ ]:
medical_df = df[(df['Einteilung (Hauptdiagnose)'] == 'Krankheit') & (df['GCS (Erstbefund)'].notna())]

In [ ]:
# use kruskal-wallis to compare GCS between different categories
from statsmodels.sandbox.stats.multicomp import MultiComparison

In [ ]:
multicomp = MultiComparison(medical_df['GCS (Erstbefund)'], medical_df['Kategorie (Hauptdiagnose)'])

In [ ]:
res = multicomp.kruskal()
res

In [ ]:
medical_df['Kategorie (Hauptdiagnose)'].value_counts()

In [ ]:
medical_df['Kategorie (Hauptdiagnose) encoded'] = medical_df['Kategorie (Hauptdiagnose)'].map(
    {'Zentralnervensystem': 0,
     'Krampfanfall': 1,
        'Herz-Kreislauf': 2,
     'Intoxikation': 3,
     'Respiratorisch': 4,
     })

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel

temp_df = medical_df[['GCS (Erstbefund)', 'Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)
mod_prob_med = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']],
                            distr='probit')

In [ ]:
res_prob_med = mod_prob_med.fit(maxiter=10000)

In [ ]:
res_prob_med.summary()

#### Trauma

In [ ]:
trauma_df = df[(df['Einteilung (Hauptdiagnose)'] == 'Unfall') & (df['GCS (Erstbefund)'].notna())]

In [ ]:
df[(df['Einteilung (Hauptdiagnose)'] == 'Unfall')]['GCS (Erstbefund)'].isna().sum()

In [ ]:
trauma_df['Kategorie (Hauptdiagnose)'].value_counts()

In [ ]:
multicomp_trauma = MultiComparison(trauma_df['GCS (Erstbefund)'], trauma_df['Kategorie (Hauptdiagnose)'])

In [ ]:
trauma_res = multicomp_trauma.kruskal()
trauma_res

In [ ]:
trauma_df['Kategorie (Hauptdiagnose) encoded'] = trauma_df['Kategorie (Hauptdiagnose)'].map(
    {'SHT': 0,
        'Polytrauma': 1,
        'Verbrennung': 2,
        'Respiratorisch': 3,
     })

In [ ]:
temp_df = trauma_df[['GCS (Erstbefund)', 'Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
temp_df.dropna(inplace=True)

mod_prob_trauma = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']],
                            distr='probit')

In [ ]:
res_prob_trauma = mod_prob_trauma.fit(maxiter=10000)

In [ ]:
res_prob_trauma.summary()

In [ ]:
import scikit_posthocs as sp

categories_gcs = [trauma_df[trauma_df['Kategorie (Hauptdiagnose)'] == cat]['GCS (Erstbefund)'] for cat in ['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch']]

dunn_res = sp.posthoc_dunn(categories_gcs, p_adjust='holm')

In [ ]:
dunn_res.columns = ['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch']
dunn_res.index = ['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch']

In [ ]:
dunn_res

Further  adjustement for AIS head

In [ ]:
trauma_df['AIS Head'].isna().sum(), trauma_df.shape

In [ ]:
# print median and IQR AIS

print(f'Overall AIS distribution: {trauma_df["AIS Head"].median(), trauma_df["AIS Head"].quantile(0.25), trauma_df["AIS Head"].quantile(0.75)}')

TBI_df = trauma_df[trauma_df['Kategorie (Hauptdiagnose)'] == 'SHT']

print(f'TBI AIS distribution: {TBI_df["AIS Head"].median(), TBI_df["AIS Head"].quantile(0.25), TBI_df["AIS Head"].quantile(0.75)}')
# number and percentage of patients with AIS head > 3 in TBI population
print(f'Number of patients with AIS head > 3 in TBI population: {len(TBI_df[TBI_df["AIS Head"] > 3])} ({len(TBI_df[TBI_df["AIS Head"] > 3]) / len(TBI_df) * 100:.2f}%)')


In [ ]:
temp_df = trauma_df[['GCS (Erstbefund)', 'Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']]
temp_df.dropna(inplace=True)

mod_prob_trauma = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['Kategorie (Hauptdiagnose) encoded', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']],
                            distr='probit')

In [ ]:
res_prob_trauma = mod_prob_trauma.fit(maxiter=10000)
res_prob_trauma.summary()

In [ ]:
res_prob_trauma.pvalues

In [ ]:
import scipy as sp

temp_df = trauma_df[['GCS (Erstbefund)', 'AIS Head']]
temp_df.dropna(inplace=True)

r, p = sp.stats.pearsonr(x=temp_df['AIS Head'], y=temp_df['GCS (Erstbefund)'])

# correlation plot of GCS and AIS
ax = sns.regplot(temp_df, x = 'AIS Head', y = 'GCS (Erstbefund)', x_jitter=.15, scatter_kws={'alpha':0.3})

# blue line handle
handle = plt.plot([],[], color=sns.color_palette("Blues")[-1], label=f'r = {r:.2f}')
plt.legend(loc='upper right')

ax.set_ylabel('GCS')

plt.show()


Seperate burn vs other categories

In [ ]:
trauma_df['burns'] = trauma_df['Kategorie (Hauptdiagnose)'].apply(lambda x: 1 if x == 'Verbrennung' else 0)

In [ ]:
trauma_df.burns.sum()

In [ ]:
temp_df = trauma_df[['GCS (Erstbefund)', 'burns', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']]
temp_df.dropna(inplace=True)
mod_prob_trauma = OrderedModel(temp_df['GCS (Erstbefund)'], temp_df[['burns', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)', 'AIS Head']],
                            distr='probit')

res_prob_trauma = mod_prob_trauma.fit(maxiter=10000)
res_prob_trauma.summary()

### Boxplots for medical and trauma

In [ ]:
# two subplots with boxplots for medical and trauma and every category

fig, axs = plt.subplots(1, 2, figsize=(15, 6))

sns.boxplot(x='Kategorie (Hauptdiagnose)', y='GCS (Erstbefund)', data=medical_df, showfliers=False, ax=axs[0], palette='mako',
            boxprops=dict(alpha=.8))
axs[0].set_title('Medical')

sns.boxplot(x='Kategorie (Hauptdiagnose)', y='GCS (Erstbefund)', data=trauma_df, showfliers=False, ax=axs[1], palette='magma',
            boxprops=dict(alpha=.8))
axs[1].set_title('Trauma')

# tilt x labels
for ax in axs:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=65)

# remove xaxis labels
axs[0].set_xlabel('')
axs[1].set_xlabel('')
plt.show()

In [ ]:
# fig.savefig('/Users/jk1/Downloads/gcs_intubation_subgroups.png', dpi=600, bbox_inches='tight')

#### table with subgroups for medical and trauma

In [ ]:
medical_df['Kategorie (Hauptdiagnose)'].replace('Zentralnvervensystem', 'Zentralnervensystem', inplace=True)
trauma_df['Kategorie (Hauptdiagnose)'].replace('Polyytrauma', 'Polytrauma', inplace=True)

In [ ]:
# build table with median (IQR) of GCS for each main diagnosis
medical_subgroups_df = pd.DataFrame(columns=['Zentralnervensystem', 'Krampfanfall', 'Herz-Kreislauf', 'Intoxikation', 'Respiratorisch'])
trauma_subgroups_df = pd.DataFrame(columns=['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch'])

medical_subgroups_df.loc['n'] = [len(medical_df[medical_df['Kategorie (Hauptdiagnose)'] == cat]) for cat in ['Zentralnervensystem', 'Krampfanfall', 'Herz-Kreislauf', 'Intoxikation', 'Respiratorisch']]
trauma_subgroups_df.loc['n'] = [len(trauma_df[trauma_df['Kategorie (Hauptdiagnose)'] == cat]) for cat in ['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch']]

for cat in ['Zentralnervensystem', 'Krampfanfall', 'Herz-Kreislauf', 'Intoxikation', 'Respiratorisch']:
    gcs_df = medical_df[medical_df['Kategorie (Hauptdiagnose)'] == cat]['GCS (Erstbefund)']
    medical_subgroups_df.loc['GCS (median/IQR)', cat] = f'{gcs_df.median()} ({gcs_df.quantile(0.25)}-{gcs_df.quantile(0.75)})'
    
for cat in ['SHT', 'Polytrauma', 'Verbrennung', 'Respiratorisch']:
    gcs_df = trauma_df[trauma_df['Kategorie (Hauptdiagnose)'] == cat]['GCS (Erstbefund)']
    trauma_subgroups_df.loc['GCS (median/IQR)', cat] = f'{gcs_df.median()} ({gcs_df.quantile(0.25)}-{gcs_df.quantile(0.75)})'


In [ ]:
medical_subgroups_df = medical_subgroups_df.T
trauma_subgroups_df = trauma_subgroups_df.T

medical_subgroups_df.reset_index(inplace=True)
trauma_subgroups_df.reset_index(inplace=True)

medical_subgroups_df.columns = ['Kategorie', 'n', 'GCS (median/IQR)']
trauma_subgroups_df.columns = ['Kategorie', 'n', 'GCS (median/IQR)']

medical_subgroups_df['Einteilung'] = 'medical'
trauma_subgroups_df['Einteilung'] = 'trauma'

medical_subgroups_df['pval'] = res_prob_med.pvalues['Kategorie (Hauptdiagnose) encoded']
trauma_subgroups_df['pval'] = res_prob_trauma.pvalues['Kategorie (Hauptdiagnose) encoded']

subgroups_df = pd.concat([medical_subgroups_df, trauma_subgroups_df])

In [ ]:
subgroups_df

In [ ]:
subgroups_df.to_csv('/Users/jk1/Downloads/gcs_intubation_subgroups.csv')

## overall model

In [ ]:
overall_temp_df = df[['GCS (Erstbefund)', 'Einteilung (Hauptdiagnose)', 'Kategorie (Hauptdiagnose)', 'Sverweis_Geschlecht_encoded', 'Alter  (Patient)']]
# rename columns to remove spaces and brackets
overall_temp_df.columns = ['GCS', 'Einteilung', 'Kategorie', 'Geschlecht', 'Alter']

In [ ]:
# patients with einteilung trauma and kategorie respiratorisch should have kategorie trauma_respiratorisch
overall_temp_df.loc[(overall_temp_df['Einteilung'] == 'Unfall') & (overall_temp_df['Kategorie'] == 'Respiratorisch'), 'Kategorie'] = 'Trauma_Respiratorisch'

In [ ]:
# numerical encoding for Eineteilung and Kategorie
overall_temp_df['Einteilung_encoded'] = overall_temp_df['Einteilung'].map({'Krankheit': 0, 'Unfall': 1})
overall_temp_df['Kategorie_encoded'] = overall_temp_df['Kategorie'].map({'Zentralnervensystem': 0,
                                                                         'Krampfanfall': 1,
                                                                         'Herz-Kreislauf': 2,
                                                                         'Intoxikation': 3,
                                                                         'Respiratorisch': 4,
                                                                         'SHT': 5,
                                                                         'Polytrauma': 6,
                                                                         'Verbrennung': 7,
                                                                         'Trauma_Respiratorisch': 8})

In [ ]:
overall_temp_df['GCS_cat'] = pd.Categorical(overall_temp_df['GCS'], categories=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=True)
# then convert to strings categories
overall_temp_df['GCS_cat'] = overall_temp_df['GCS_cat'].astype(str)
# and then convert to strings categories
overall_temp_df['GCS_cat'] = pd.Categorical(overall_temp_df['GCS_cat'], categories=['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], ordered=True)

In [ ]:
overall_temp_df.dropna(inplace=True)

In [ ]:
overall_temp_df 

In [ ]:
import os
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"
from pymer4.models import Lmer

# model = Lmer(f"GCS ~ Kategorie_encoded + Geschlecht + Alter + (1|Einteilung_encoded)",
#                          data=overall_temp_df, family='binomial')
model = Lmer(f"GCS_cat ~ Kategorie_encoded + Geschlecht + Alter + (1|Einteilung_encoded)", data=overall_temp_df, family='binomial')
model.fit()